In [162]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [163]:
train_data = pd.read_excel(r'C:\Users\harsh\Downloads/Data_Train.xlsx')


In [164]:
test_data = pd.read_excel(r'C:\Users\harsh\Downloads/Test_set.xlsx')

In [165]:
train_data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [166]:
#check for missing values
train_data.isnull().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of        Airline  Date_of_Journey  Source  Destination  Route  Dep_Time  \
0        False            False   False        False  False     False   
1        False            False   False        False  False     False   
2        False            False   False        False  False     False   
3        False            False   False        False  False     False   
4        False            False   False        False  False     False   
...        ...              ...     ...          ...    ...       ...   
10678    False            False   False        False  False     False   
10679    False            False   False        False  False     False   
10680    False            False   False        False  False     False   
10681    False            False   False        False  False     False   
10682    False            False   False        False  False     False   

       Arrival_Time  Duration  Total_Stops  Additional_Info  

In [167]:
train_data.isna().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [168]:
train_data.shape

(10683, 11)

In [169]:
train_data.dropna(inplace = True)

In [170]:
train_data.isna().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
dtype: int64

# Clean the Data for Modeling

In [171]:
# function to change necessary columns from object to date time
def change_to_time(column):
    train_data[column] = pd.to_datetime(train_data[column])
    
    

In [172]:
for i in ['Date_of_Journey', 'Dep_Time', 'Arrival_Time']:
    change_to_time(i)

In [173]:
train_data.dtypes

Airline                    object
Date_of_Journey    datetime64[ns]
Source                     object
Destination                object
Route                      object
Dep_Time           datetime64[ns]
Arrival_Time       datetime64[ns]
Duration                   object
Total_Stops                object
Additional_Info            object
Price                       int64
dtype: object

In [174]:
# create a new column to add a day of journey 
train_data['Journey_Day'] = train_data['Date_of_Journey'].dt.day

In [175]:
# create a new column to add a month of journey 
train_data['Journey_Month'] = train_data['Date_of_Journey'].dt.month

In [176]:
#function to get hour of departure or arrival time for each flight
def get_hour(df, col):
    df[col + '_hour'] = df[col].dt.hour
    #function to get hour of departure or arrival time for each flight
def get_min(df, col):
    df[col + '_min'] = df[col].dt.minute


In [177]:
get_hour(train_data, 'Dep_Time')
get_min(train_data, 'Dep_Time')
train_data.drop('Dep_Time', axis = 1, inplace = True)

In [178]:
get_hour(train_data, 'Arrival_Time')
get_min(train_data, 'Arrival_Time')
train_data.drop('Arrival_Time', axis = 1, inplace = True)

In [179]:
# Split the duration into hour in minute
duration=list(train_data['Duration'])
for i in range(len(duration)):
    if len(duration[i].split(' '))==2:
        pass
    else:
        if 'h' in duration[i]: # Check if duration contains only hour
             duration[i]=duration[i] + ' 0m' # Adds 0 minute
        else:
             duration[i]='0h '+ duration[i]
    
    

In [180]:
train_data['Duration'] = duration

In [181]:
def hour(hr):
    return hr.split(' ')[0][0:-1]
def minute(hr):
    return hr.split(' ')[1][0:-1]

In [182]:
# Creating new columns Duration Hr and Duration Min to create separate values 
train_data['Duration_Hour'] = train_data['Duration'].apply(hour)
train_data['Duration_Minute'] = train_data['Duration'].apply(minute)

In [193]:
train_data.dtypes

Airline                      object
Date_of_Journey      datetime64[ns]
Source                       object
Destination                  object
Route                        object
Total_Stops                  object
Additional_Info              object
Price                         int64
Journey_Day                   int64
Journey_Month                 int64
Dep_Time_hour                 int64
Dep_Time_min                  int64
Arrival_Time_hour             int64
Arrival_Time_min              int64
Duration_Hour                 int32
Duration_Minute               int32
dtype: object

In [184]:
train_data.drop('Duration', axis = 1, inplace = True)

In [189]:
train_data['Duration_Hour'] = train_data['Duration_Hour'].astype(str).astype(int)
train_data['Duration_Minute'] = train_data['Duration_Minute'].astype(str).astype(int)

In [195]:
# if the type is an Object('O'), a
catCol = [col for col in train_data.columns if train_data[col].dtype == 'O']
catCol    

['Airline', 'Source', 'Destination', 'Route', 'Total_Stops', 'Additional_Info']

In [197]:
# if the type is not an Object('O'), a
numCol = [col for col in train_data.columns if train_data[col].dtype != 'O']
numCol  

['Date_of_Journey',
 'Price',
 'Journey_Day',
 'Journey_Month',
 'Dep_Time_hour',
 'Dep_Time_min',
 'Arrival_Time_hour',
 'Arrival_Time_min',
 'Duration_Hour',
 'Duration_Minute']

In [190]:
train_data.dtypes

Airline                      object
Date_of_Journey      datetime64[ns]
Source                       object
Destination                  object
Route                        object
Total_Stops                  object
Additional_Info              object
Price                         int64
Journey_Day                   int64
Journey_Month                 int64
Dep_Time_hour                 int64
Dep_Time_min                  int64
Arrival_Time_hour             int64
Arrival_Time_min              int64
Duration_Hour                 int32
Duration_Minute               int32
dtype: object

In [191]:
train_data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Total_Stops,Additional_Info,Price,Journey_Day,Journey_Month,Dep_Time_hour,Dep_Time_min,Arrival_Time_hour,Arrival_Time_min,Duration_Hour,Duration_Minute
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,non-stop,No info,3897,24,3,22,20,1,10,2,50
1,Air India,2019-01-05,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,No info,7662,5,1,5,50,13,15,7,25
2,Jet Airways,2019-09-06,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,No info,13882,6,9,9,25,4,25,19,0
3,IndiGo,2019-12-05,Kolkata,Banglore,CCU → NAG → BLR,1 stop,No info,6218,5,12,18,5,23,30,5,25
4,IndiGo,2019-01-03,Banglore,New Delhi,BLR → NAG → DEL,1 stop,No info,13302,3,1,16,50,21,35,4,45
